# Explainers Notebook

 ## Motivation 

### What is your dataset?

    Star Wars Movie Transcipts for the episodes
    Wookipedia pages for the relevant characters
    

### Why did you choose this/these particular dataset(s)?

    The transcript as they hold all the information given in the movie
    Wookieepedia pages because they hold data and connections between the characters
    
### What was your goal for the end user's experience?

    To give an insight to what might not be obvious in the universe and get another understanding of the universe.

## The Dataset
 
### Write about your choices in data cleaning and preprocessing

    Used regex to get the name from the list on wikipedia. All character names was written {{visible anchor | name}}, except the one for Anakin Skywalker/Darth Vader. So we could easily write a regular expression to get these. However not all the names, was the same as the ones used on Wookiepedia, which mean we had to cut of some titles like Admiral from the Wiki name or only use the last name. This however resulted in getting to many names, as some of the last name of characters had their own Wookiedpedia pages such as (Jabba the) Hutt. When this was cleaned up manually, we ended up with the 409 characters we will use for the network analysis. These names have then for convinence been saved in a CSV file along with goodness and species. We can then easily download the wookieepedia pages for all the characters. The csv file, and character pages can be found link or downloaded using this code
    
    The movie transcript was found on IMSDB, we then used regex to only get the script part and file can be found link or download using the code.

### Write a short section that discusses the dataset stats

 ## Tools, theory and analysis
 
     Talk about how you've worked with text, including regular expressions, unicode, etc.
 
     Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.
     
    How did you use the tools to understand your dataset?

 
 ### Explain the overall idea
 

 

 

 ### Analysis step 1
 
 #### Explain what you're interested in
     
     Interested in creating a network over the characters, to analyze connections to see who is the most important person, analyze communities to see if these follow any grouping of the actual stuff such as race, movie, alliance, family or other.
 
 #### Explain the tool
 
 #### Apply the tool
 
 #### Discuss the outcome

 ### Analysis step 2
 
 #### Explain what you're interested in
 
 #### Explain the tool
 
 #### Apply the tool
 
 #### Discuss the outcome

 ## Discussion 
 